#FloPy3
##Parameter Estimation with FloPy
This notebook demonstrates the current parameter estimation functionality that is available with FloPy.  The capability to write a simple template file for PEST is the only capability implemented so far.  The plan is to develop functionality for creating PEST instruction files as well as the PEST control file.

In [1]:
%matplotlib inline
import numpy as np
import flopy
import flopy.pest.templatewriter as tplwriter
import flopy.pest.params as params

This notebook will work with a simple model using the dimensions below

In [2]:
# Define the model dimensions
nlay = 3
nrow = 20
ncol = 20

# Create the flopy model object and add the dis and lpf packages
m = flopy.modflow.Modflow(modelname='mymodel', model_ws='./data')
dis = flopy.modflow.ModflowDis(m, nlay, nrow, ncol)
lpf = flopy.modflow.ModflowLpf(m, hk=10.)

### Simple One Parameter Example
In order to create a PEST template file, we first need to define a parameter.  For example, let's say we want to parameterize hydraulic conductivity.  As a first step, let's define a parameter called HK_LAYER_1 and assign it to all of layer 1.  We will not parameterize hydraulic conductivity for layers 2 and 3 and instead leave HK at its value of 10. (as assigned in the block above this one). We can do this as follows.

In [3]:
mfpackage = 'lpf'
partype = 'hk'
parname = 'HK_LAYER_1'
startvalue = 10.
lbound = 0.001
ubound = 1000.
idx = np.empty((nlay, nrow, ncol), dtype=np.bool)
idx[0] = True
idx[1:] = False
transform='log'

p = params.Params(mfpackage, partype, parname, startvalue, 
                  lbound, ubound, idx)

At this point, we have enough information to the write a PEST template file for the LPF package.  We can do this using the following statement:

In [4]:
tw = tplwriter.TemplateWriter(m, [p])
tw.write_template()

At this point, the lpf template file will have been created.  The following block will print the template file.

In [5]:
lines = open('./data/mymodel.lpf.tpl', 'r').readlines()
for l in lines:
    print(l.strip())

ptf ~
# LPF for MODFLOW, generated by Flopy.
53    -1E+30         0
0         0         0
0         0         0
1.000000E+00   1.000000E+00   1.000000E+00
0         0         0
0         0         0
CONSTANT ~ HK_LAYER_1  ~    #hk
CONSTANT    1.000000E+00                           #vka Layer 1
CONSTANT 10.0    #hk
CONSTANT    1.000000E+00                           #vka Layer 2
CONSTANT 10.0    #hk
CONSTANT    1.000000E+00                           #vka Layer 3


###Multiple Parameter Zoned Approach

The params module has a helper function called zonearray2params that will take a zone array and some other information and create a list of parameters, which can then be passed to the template writer.  This next example shows how to create a slightly more complicated LPF template file in which both HK and VKA are parameterized.

In [6]:
# Create a zone array
zonearray = np.ones((nlay, nrow, ncol), dtype=int)
zonearray[0, 10:, 7:] = 2
zonearray[0, 15:, 9:] = 3
zonearray[1] = 4

In [7]:
# Create a list of parameters for HK
mfpackage = 'lpf'
parzones = [2, 3, 4]
parvals = [56.777, 78.999, 99.]
lbound = 5
ubound = 500
transform = 'log'
plisthk = params.zonearray2params(mfpackage, 'hk', parzones, lbound, 
                                  ubound, parvals, transform, zonearray)

In this case, Flopy will create three parameters: hk_2, hk_3, and hk_4, which will apply to the horizontal hydraulic conductivity for cells in zones 2, 3, and 4, respectively.  Only those zone numbers listed in parzones will be parameterized.  For example, many cells in zonearray have a value of 1.  Those cells will not be parameterized.  Instead, their hydraulic conductivity values will remain fixed at the value that was specified when the Flopy LPF package was created.

In [8]:
# Create a list of parameters for VKA
parzones = [1, 2]
parvals = [0.001, 0.0005]
zonearray = np.ones((nlay, nrow, ncol), dtype=int)
zonearray[1] = 2
plistvk = params.zonearray2params(mfpackage, 'vka', parzones, lbound, 
                                  ubound, parvals, transform, zonearray)

In [9]:
# Combine the HK and VKA parameters together
plist = plisthk + plistvk
for p in plist:
    print(p.name, p.mfpackage, p.startvalue)

('hk_2', 'lpf', 56.777)
('hk_3', 'lpf', 78.999)
('hk_4', 'lpf', 99.0)
('vka_1', 'lpf', 0.001)
('vka_2', 'lpf', 0.0005)


In [10]:
# Write the template file
tw = tplwriter.TemplateWriter(m, plist)
tw.write_template()

In [11]:
# Print contents of template file
lines = open('./data/mymodel.lpf.tpl', 'r').readlines()
for l in lines:
    print(l.strip())

ptf ~
# LPF for MODFLOW, generated by Flopy.
53    -1E+30         0
0         0         0
0         0         0
1.000000E+00   1.000000E+00   1.000000E+00
0         0         0
0         0         0
INTERNAL 1.0 (FREE) -1      #hk
10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0
10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0
10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0
10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0
10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0            10.0
10.0            10.0    